<a href="https://www.kaggle.com/code/jagadeeshjr5/gemma-quantization?scriptVersionId=178323067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

import os

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
from huggingface_hub import login
login()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    torch_dtype=torch.float32, device_map='cuda'
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
torch.save(model.state_dict(), '/kaggle/working/gemma-2b-it.pth')

In [5]:
import numpy as np

def scale(rmax, rmin, bits=8, mode='affine'):
    if mode == 'affine':
        qmax = 2**(bits - 1) - 1
        qmin = -2**(bits - 1)
        s = (rmax - rmin) / (qmax - qmin)
    else:
        s = rmax / 2**(bits - 1)
    return s

def zero_point(s, rmin, bits=8, mode='affine'):
    if mode == 'affine':
        qmax = 2**(bits - 1) - 1
        qmin = -2**(bits - 1)
        z = np.round(qmin - (rmin / s))
    else:
        z = 0
    return z

def quantize(x, bits=8, mode='affine', dtype=np.int8):
    s = scale(x.max(), x.min(), bits)
    z = zero_point(s, x.min(), bits)
    qmax = 2**(bits - 1) - 1
    qmin = -2**(bits - 1)
    quantized_x = np.round(1 / s * x + z, decimals=0)
    quantized_x = np.clip(quantized_x, a_min=qmin, a_max=qmax)
    quantized_x = quantized_x.astype(dtype)
    return quantized_x

def dequantize(qx, rmax, rmin, bits=8, mode='affine', dtype=np.float32):
    s = scale(rmax, rmin, bits)
    z = zero_point(s, rmin, bits)
    qx = qx.astype(np.int32)
    dequantized_x = s * (qx - z)
    dequantized_x = dequantized_x.astype(dtype)
    return dequantized_x

In [6]:
quantized_model_parameters = {}
minmax_values = {}

for key, value in model.state_dict().items():
    value_np = value.cpu().numpy()
    quantized_value_np = quantize(value_np)
    quantized_value_tensor = torch.from_numpy(quantized_value_np)
    quantized_model_parameters[key] = quantized_value_tensor
    minmax_values[key] = [value.max().item(), value.min().item()]

In [7]:
torch.save(quantized_model_parameters, "gemma-2b-linearquantized.pth")
torch.save(minmax_values, "gemma-2b-minmax-parameters.pth")

In [8]:
print('gemma-2b-it size (GB):', np.round(os.path.getsize("/kaggle/working/gemma-2b-it.pth")/1e9), 2)
print('Quantized gemma-2b-it size (GB):', np.round(os.path.getsize("/kaggle/working/gemma-2b-linearquantized.pth")/1e9), 2)

gemma-2b-it size (GB): 10.0 2
Quantized gemma-2b-it size (GB): 3.0 2


In [9]:
assert model.state_dict().keys() == torch.load('/kaggle/working/gemma-2b-linearquantized.pth', map_location=torch.device('cpu')).keys(), "State dicts have different keys!"

mse_losses = []

model.to('cpu')

for key in tqdm(model.state_dict().keys(), total=len(model.state_dict().keys())):
    weight1 = model.state_dict()[key]
    weight2 = dequantize(torch.load('/kaggle/working/gemma-2b-linearquantized.pth', map_location=torch.device('cpu'))[key].numpy(), torch.load('/kaggle/working/gemma-2b-minmax-parameters.pth', map_location=torch.device('cpu'))[key][0], torch.load('/kaggle/working/gemma-2b-minmax-parameters.pth', map_location=torch.device('cpu'))[key][1])
    weight2 = torch.tensor(weight2, dtype=weight1.dtype)
    mse_loss = F.mse_loss(weight1, weight2)
    mse_losses.append(mse_loss.item())
    

average_mse_loss = sum(mse_losses) / len(mse_losses)

#print("Individual MSE losses:", mse_losses)
print("Average MSE loss:", average_mse_loss)

100%|██████████| 165/165 [05:29<00:00,  2.00s/it]

Average MSE loss: 2.1192357436819715e-05
